# Agent Engine の A2A 対応機能の利用例 (2)

Agent Engine に A2A サーバーの機能がデプロイできるようになりました。

たとえば、下記の記事では、Agent Engine にデプロイした ADK Agent の前段に Cloud Run を用いて A2A サーバーを用意していましたが、Agent Engine に A2A サーバーの機能も同時にデプロイできるようになります。

- [A2A リモートエージェント対応のマルチエージェントシステム](https://zenn.dev/google_cloud_jp/articles/d24a22a87a84f8)

このノートブックでは、上記の記事で Cloud Run にデプロイしていた A2A サーバーの機能を Agent Engine に移行する例を紹介します。

この例では、ADK が提供する A2A 対応モジュールを利用して、ADK の標準的なコードで実装できるようにしてあります。

## パッケージインストール

In [ ]:
%pip install --user \
    google-adk==1.14.1 \
    google-genai==1.36.0 \
    google-cloud-aiplatform==1.113.0 \
    a2a-sdk==0.3.5

カーネルを再起動します。

In [3]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## 事前準備

In [2]:
import copy, json, os, uuid
import vertexai
from vertexai import agent_engines

from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.events import Event
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

root agent をローカルで実行する簡易アプリのクラスを定義します。

In [3]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## リモートエージェントの定義

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [6]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [7]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

## Agent Engine にデプロイ

In [8]:
from a2a.types import AgentCard
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.auth.credential_service.in_memory_credential_service import InMemoryCredentialService
from google.adk.a2a.utils.agent_card_builder import AgentCardBuilder

from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card

from google.adk.sessions import VertexAiSessionService

# Wrapper class of VertexAiSessionService that's compatible with InMemorySessionService
class MyVertexAiSessionService(VertexAiSessionService):
    def __init__(self, resource_id, project=PROJECT_ID, location=LOCATION):
        super().__init__(project, location)
        self.app_name=resource_id

    async def list_sessions(self, app_name, user_id):
        return await super().list_sessions(
            app_name=self.app_name,
            user_id=user_id
        )

    async def create_session(self, app_name, user_id, state={}, session_id=None):
        session = await super().create_session(
            app_name=self.app_name,
            user_id=user_id,
            state=state,
        )
        return session

    async def get_session(self, app_name, user_id, session_id):
        try:
            session = await super().get_session(
                app_name=self.app_name,
                user_id=user_id,
                session_id=session_id,
            )
            return session
        except:
            return None


def get_create_runner_class(agent, resource_name):
    async def create_runner():
        return Runner(
            app_name=agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=MyVertexAiSessionService(resource_name),
            memory_service=InMemoryMemoryService(),
            credential_service=InMemoryCredentialService(),
        )
    return create_runner


def get_agent_executor_class(agent, resource_name):
    def agent_executor_builder():
        return A2aAgentExecutor(
            runner=get_create_runner_class(agent, resource_name),
        )       
    return agent_executor_builder


async def get_agent_card(agent):
    builder = AgentCardBuilder(agent=agent)
    adk_agent_card = await builder.build()
    return create_agent_card(
        agent_name=adk_agent_card.name,
        description=adk_agent_card.description,
        skills=adk_agent_card.skills
    )

ここでは、一度、ダミーでデプロイして VertexAiSessionService で必要となる Agent Engine のリソース ID を取得した後、リソース ID を指定して再デプロイしています。

In [11]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None

In [91]:
agents = {
    'research_agent1_remote': research_agent1,
    'research_agent2_remote': research_agent2,
    'writer_agent_remote': writer_agent,
    'review_agent_remote': review_agent,
}

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version="v1beta1", base_url=f"https://{LOCATION}-aiplatform.googleapis.com/"
    ),
)

for agent_name, agent in agents.items():    
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, None)
    )
    config={
        "display_name": agent_name,
        "description": a2a_agent.agent_card.description,
        "requirements": [
            'google-adk==1.14.1',
            'google-genai==1.36.0',
            'google-cloud-aiplatform==1.113.0',
            'a2a-sdk==0.3.5'
        ],
        "http_options": {
            "base_url": f"https://{LOCATION}-aiplatform.googleapis.com",
            "api_version": "v1beta1",
        },
        "staging_bucket": f'gs://{PROJECT_ID}',
    }

    resource_name = get_agent_resource(agent_name)
    if not resource_name:
        # Deploy dummy agent to assign Agent Engine resource name
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    
    # Redeploy with VertexAiSessionService
    resource_name = get_agent_resource(agent_name)
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, resource_name)
    )
    remote_a2a_agent = client.agent_engines.update(
        name=resource_name,
        agent=a2a_agent,
        config=config,
    )

/var/tmp/ipykernel_11563/1362738177.py:71: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)
The following requirements are missing: {'cloudpickle', 'pydantic', 'a2a'}
/var/tmp/ipykernel_11563/1362738177.py:71: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)
The following requirement

## RemoteA2aAgent クラスのインスタンスを作成

In [12]:
import os
import httpx
from a2a.client import ClientConfig, ClientFactory
from a2a.types import TransportProtocol
from google.adk import Agent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.auth import default
from google.auth.transport.requests import Request


credentials, _ = default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
credentials.refresh(Request())
factory = ClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
        httpx_client=httpx.AsyncClient(
            timeout=60, # This is important.
            headers={
                "Authorization": f"Bearer {credentials.token}",
                "Content-Type": "application/json",
            }
        ),
    )
)

resource_name = get_agent_resource('research_agent1_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
research_agent1_remote = RemoteA2aAgent(
    name="research_agent1",
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
research_agent2_remote = RemoteA2aAgent(
    name="research_agent2",
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
writer_agent_remote = RemoteA2aAgent(
    name="writer_agent",
    description='特定のテーマに関する読み物記事を書くエージェント',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
review_agent_remote = RemoteA2aAgent(
    name="review_agent",
    description='読み物記事をレビューするエージェント',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

/var/tmp/ipykernel_78673/2722862927.py:29: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent1_remote = RemoteA2aAgent(
/var/tmp/ipykernel_78673/2722862927.py:38: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent2_remote = RemoteA2aAgent(
/var/tmp/ipykernel_78673/2722862927.py:47: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementat

## Root agent の定義

In [13]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        research_agent1_remote,
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        research_agent2_remote,
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        writer_agent_remote,
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        review_agent_remote,
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

In [14]:
root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [25]:
client = LocalApp(root_agent)
DEBUG = False

query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は、ユーザーさんが指定したテーマの記事を作成するお手伝いをすることができます。

まず、リサーチエージェントがテーマに関する調査レポートを作成します。その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。ご希望のテーマがあれば教えてください！



In [26]:
query = '''
「夏の節電対策」をテーマに記事を作成してください。
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


承知いたしました。「夏の節電対策」をテーマにした記事の作成に取り掛かります。まずは、リサーチエージェントに調査レポートを依頼します。



---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

承知いたしました。「夏の節電対策」に関する記事作成のための調査レポート作成に向け、以下のトピックを提案します。

### 調査対象トピック

1.  **夏の節電が求められる背景と重要性**:
    *   電力需給のひっ迫状況や電気料金の高騰傾向。
    *   地球温暖化対策としての節電の意義。
    *   夏の消費電力の内訳と特徴。
2.  **家庭で実践できる具体的な節電対策（家電別）**:
    *   エアコンの効率的な使い方（設定温度、併用家電、フィルター清掃など）。
    *   冷蔵庫、照明、テレビ、PCなど、その他の主要家電の節電術。
    *   省エネ家電への買い替えメリットと選び方。
3.  **住まいの構造・生活習慣からの節電対策**:
    *   窓や屋根の遮熱・断熱対策（遮光カーテン、すだれ、グリーンカーテンなど）。
    *   日中の外出、夕涼みなど、生活リズムと節電の関連性。
    *   調理、入浴時の節電工夫。
4.  **節電によるメリットと注意点**:
    *   電気代節約効果の試算例。
    *   環境負荷低減への貢献。
    *   熱中症予防と節電の両立方法、健康を損なわないための注意点。
5.  **国の施策・自治体や電力会社の取り組み、最新技術**:
    *   国の省エネ推進策や補助金制度。
    *   自治体や電力会社が提供する節電プログラムやポイント制度。
    *   HEMS（ヘムス）やスマート家電など、最新技術を活用した節電方法。

## 選定したトピックに基づいて、調査レポートを作成します。

## 夏の節電対策に関する調査レポート

### 1. 夏の節電が求められる背景と重要性

近年、特に夏季は電力需要がピークを迎えやすく、供給予備率の低下から電力需給ひっ迫警報が発令されるケースが増加しています。さらに、国際情勢や燃料費の高騰により電気料金が上昇傾向にあり、家庭や企業における電気代の負担が増大しています。電力の安定供給を確保し、大規模な停電を回避するためにも、社会全体での節電意識の向上が不可欠です。また、化石燃料に依存する発電は温室効果ガスを排出し、地球温暖化を加速させるため、電力消費を抑えることは脱炭素社会の実現に向けた重要な貢献となります。夏の

In [27]:
query = '''
お願いします。
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


承知いたしました。調査レポートに基づき、ライターエージェントとレビューエージェントに記事の作成とレビューを依頼します。



---
## ライターエージェントが記事を執筆します。
---



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

今年の夏も猛暑が予想されますが、同時に気になるのが電気代や電力需給。賢く、そして快適に夏を乗り切るための節電術を一緒に見ていきましょう！

## 夏の節電、なぜ大切？

「夏の節電」と聞くと、「我慢しなきゃいけないの？」と思う方もいるかもしれませんね。でも実は、単に電気代を安くするだけでなく、もっと大きな理由があるんです。近年、夏は電力の需要がグッと高まり、時に電力需給ひっ迫警報が出るほど。これは、大規模な停電につながる可能性もあるため、私たち一人ひとりの節電が、電力の安定供給を支える大切な行動になります。しかも、電気を作るために使う燃料は高騰中。電気代の値上がりは家計を直撃しています。さらに、電力消費を抑えることは、地球温暖化の原因となる二酸化炭素の排出量を減らすことにもつながる、まさに一石三鳥の取り組みなんです。夏の家庭の消費電力はエアコンが半分以上を占めていることが多いので、ここが節電の大きなポイントになりますよ。

## 最新技術で賢く節電！

「節電って面倒…」そんな風に感じている方に朗報です！最近は、最新の技術を活用して、快適さを保ちながら賢く節電できる方法が増えています。例えば、「HEMS（Home Energy Management System）」は、家庭内のエネルギー使用量をリアルタイムで「見える化」してくれるシステム。どこでどれだけ電気を使っているかが一目瞭然になるので、無駄遣いを特定しやすくなります。さらに、AIを搭載したエアコンのようなスマート家電は、部屋の状況や人の動きを感知して、自動で最適な運転をしてくれる優れもの。自分で細かく設定しなくても、効率的に節電してくれるので、手間なく快適に過ごせます。国や自治体も、省エネ家電への買い替え補助金制度を用意していることが多いので、これを機に最新家電の導入を検討してみるのも良いでしょう。

## 今日からできる！夏の節電アクション

「よし、やってみよう！」と思ったあなたに、今日からすぐに実践できる節電対策をいくつかご紹介します。

*   **エアコンの設定温度を見直そう**: 夏のエアコンは28℃を目安に設定しましょう。たった1℃上げるだけでも約10%の節電効果があると言われています。
*   **扇風機やサーキュレーターを併用**: エアコンと組み合わせることで、冷たい空気を効率よく循環させ

In [28]:
query = '''
お願いします。
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


承知いたしました。レビューエージェントからの指摘に基づき、記事の修正をライターエージェントに依頼します。



---
## ライターエージェントが記事を執筆します。
---



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

# 夏の賢い節電術で快適＆エコライフ🌻

今年の夏も猛暑が予想され、電力ひっ迫のニュースが増えています。家計にも環境にも優しい、賢い節電術を今こそ実践してみませんか？

## 概要：夏の節電、なぜ大切？💡

夏の節電は、単に電気代を安くするだけでなく、日本の電力供給を安定させ、地球温暖化対策にも貢献する大切な行動です。特に夏はエアコンの使用が増えるため電力需要が跳ね上がり、供給が追いつかなくなるリスクが高まります。私たち一人ひとりの小さな工夫が、大きな力となるのです。無理なく賢く節電を実践することで、快適な夏を過ごしながら、持続可能な社会づくりにも貢献できます。

## 最新情報：進化する節電テクノロジー🤖

最近では、AIやIoTを活用した新しい節電技術が注目されています。例えば、「HEMS（ヘムス）」と呼ばれる家庭内のエネルギー管理システムは、家電の電力使用量を「見える化」し、自動で最適な運転をサポートします。また、スマートエアコンやスマート照明といった「スマート家電」は、スマホアプリと連携して外出先からの操作やスケジュール設定が可能。人がいない時は自動で電源をオフにしたり、時間帯によって設定を変えたりすることで、無駄な電力消費を大幅に削減できます。これらの技術を上手に活用すれば、快適さを損なわずに効率的な節電が実現可能です。

## 実践：今日からできる！夏の節電アクション5選✨

ここでは、今日からすぐに始められる具体的な節電アクションを5つご紹介します！

*   **エアコンの設定温度を見直す:** 冷房時の設定温度を1℃上げるだけで、約10%の節電効果があると言われています。扇風機やサーキュレーターと併用して、体感温度を下げましょう。
*   **定期的なフィルター掃除:** エアコンのフィルターが汚れていると、冷暖房効率が低下し、余分な電力を消費します。2週間に1度を目安に掃除するだけで、年間を通して大きな節電効果が期待できます。
*   **窓からの日差しを遮る:** 窓から入る日差しは、室温上昇の大きな原因です。遮光カーテンやすだれ、ブラインドなどを利用して日差しを遮ることで、エアコンの負荷を減らせます。
*   **冷蔵庫の使い方を見直す:** 冷蔵庫に食品を詰め込みすぎると、冷気の循環が悪くなり、余分な電力を消費します。また、開閉回数を減らし

In [29]:
query = '''
これで完成です。全文を表示してください。
'''
result = await client.stream(query)

# 電気代高騰ストップ！エアコン快適＆家計に優しい夏の節電術🌻

今年の夏も猛暑が予想され、電力ひっ迫のニュースが増えています。家計にも環境にも優しい、賢い節電術を今こそ実践してみませんか？

## 概要：夏の節電、なぜ大切？💡

夏の節電は、単に電気代を安くするだけでなく、日本の電力供給を安定させ、地球温暖化対策にも貢献する大切な行動です。特に夏はエアコンの使用が増えるため電力需要が跳ね上がり、供給が追いつかなくなるリスクが高まります。私たち一人ひとりの小さな工夫が、大きな力となるのです。無理なく賢く節電を実践することで、快適な夏を過ごしながら、持続可能な社会づくりにも貢献できます。

## 最新情報：進化する節電テクノロジー🤖

最近では、AIやIoTを活用した新しい節電技術が注目されています。例えば、「HEMS（ヘムス）」と呼ばれる家庭内のエネルギー管理システムは、家電の電力使用量を「見える化」し、自動で最適な運転をサポートします。また、スマートエアコンやスマート照明といった「スマート家電」は、スマホアプリと連携して外出先からの操作やスケジュール設定が可能。人がいない時は自動で電源をオフにしたり、時間帯によって設定を変えたりすることで、無駄な電力消費を大幅に削減できます。これらの技術を上手に活用すれば、快適さを損なわずに効率的な節電が実現可能です。

## 実践：今日からできる！夏の節電アクション5選✨

ここでは、今日からすぐに始められる具体的な節電アクションを5つご紹介します！

*   **エアコンの設定温度を見直す:** 冷房時の設定温度を1℃上げるだけで、約10%の節電効果があると言われています。扇風機やサーキュレーターと併用して、体感温度を下げましょう。
*   **定期的なフィルター掃除:** エアコンのフィルターが汚れていると、冷暖房効率が低下し、余分な電力を消費します。2週間に1度を目安に掃除するだけで、年間を通して大きな節電効果が期待できます。
*   **窓からの日差しを遮る:** 窓から入る日差しは、室温上昇の大きな原因です。遮光カーテンやすだれ、ブラインドなどを利用して日差しを遮ることで、エアコンの負荷を減らせます。
*   **冷蔵庫の使い方を見直す:** 冷蔵庫に食品を詰め込みすぎると、冷気の循環が悪くなり、余分な電力を消費します